In [4]:
import pandas as pd
import re
import sqlite3
import cufflinks as cf
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
# Configuration
home_path = "./../../"
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Loading data

In [6]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect(home_path+"collectors/data.sqlite3")
df = pd.read_sql_query("SELECT * from job_post", con)
con.close()
df.head(5)

id                                 title                   company  \
0   1                        Data Scientist       Aquatic Informatics   
1   2         Business Intelligence Analyst                   GLENTEL   
2   3        Human Resources Data Scientist                 Rio Tinto   
3   4  Lead - Human Resource Data Scientist                 Rio Tinto   
4   5             Machine Learning Engineer  Skycope Technologies Inc   

        location                                        description  \
0  Vancouver, BC  Do you want a meaningful role in a company tha...   
1    Burnaby, BC  Brand: Glentel Corporate\nLocation: Burnaby Of...   
2         Canada  2 x newly created Data Scientist opportunities...   
3         Canada  Newly created data science lead embedded withi...   
4  Vancouver, BC  Who We are\nFounded in 2016, Skycope Technolog...   

       source       search_kw  
0  indeed.com  data scientist  
1  indeed.com  data scientist  
2  indeed.com  data scientist  
3  indeed.com  data scientist  
4  indeed.com  data scientist

In [11]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://postgres@localhost:5432/postgres', echo=False)
df[['title','company','location','description','source','search_kw']].to_sql('job_post', con=engine, if_exists='append', index=False)

In [16]:
con = sqlite3.connect(home_path+"collectors/data.sqlite3")
df = pd.read_sql_query("SELECT * from company_review", con)
con.close()
df.head(5)

id              company                                              title  \
0   1  bmo financial group                                             Decent   
1   2  bmo financial group                                 easy place to work   
2   3  bmo financial group  Dead Industry- They will squeeze you like a le...   
3   4  bmo financial group  In BMO agile team, one developer has to servic...   
4   5  bmo financial group                                               Good   

  rating                                    author author_status  \
0    3.0        Credit Analyst (Current Employee)            N/A   
1    3.0        Project Manager (Former Employee)            N/A   
2    1.0      Assistant Manager (Former Employee)            N/A   
3    2.0    Software Specialist (Former Employee)            N/A   
4    5.0  Relationship Manager (Current Employee)            N/A   

          location           date  \
0     Burnaby, BC    11 July 2018   
1     Toronto, ON    7 April 2020   
2   Cambridge, ON    6 April 2020   
3     Toronto, ON    6 April 2020   
4     Nanaimo, BC    4 April 2020   

                                         description      source  
0  Not a bad job but can get boring easily. Manag...  indeed.com  
1  co-workers are friendly but pay is not good at...  indeed.com  
2  They give you the illusion of how great of a j...  indeed.com  
3  Basically, in BMO agile team, they hire 5+ per...  indeed.com  
4  Work/life balance and customer centric. Traini...  indeed.com

In [34]:
df = pd.read_sql_query('select * from job_post',engine)
df.head(5)

id                          title                             company  \
0   1      Machine Learning Engineer            Skycope Technologies Inc   
1   2  Senior Software Engineer - C#  Wenco International Mining Systems   
2   3                 Data Processor            Westland Insurance Group   
3   4        E-Commerce Merchandiser                         Kit and Ace   
4   5        Lead Big Data Developer                          Maarut Inc   

              location                                        description  \
0        Vancouver, BC  Who We are\nFounded in 2016, Skycope Technolog...   
1         Richmond, BC  Who We are\nFounded in 2016, Skycope Technolog...   
2  New Westminster, BC  Westland Insurance Group Ltd is growing and ha...   
3        Vancouver, BC  E-Commerce Merchandiser\nVancouver, BC\nWe are...   
4        Vancouver, BC  Responsibilities\nExtensive experience in data...   

       source         search_kw  
0  indeed.com  machine learning  
1  indeed.com  machine learning  
2  indeed.com          big data  
3  indeed.com          big data  
4  indeed.com          big data

In [22]:
def extr1(auth):
    indx1 = auth.index('(')
    indx2 = auth.index(')')
    job_title = auth[0:indx1]
    job_status = auth[indx1+1:indx2]
    return job_title

def extr2(auth):
    indx1 = auth.index('(')
    indx2 = auth.index(')')
    job_title = auth[0:indx1]
    job_status = auth[indx1+1:indx2]
    return job_status

df['author'] = df.apply(lambda row: extr1(row.author), axis=1)
df.head(5)
# df['author'] = author[0]
# df['author_status'] = author[1]

id              company                                              title  \
0   1  bmo financial group                                             Decent   
1   2  bmo financial group                                 easy place to work   
2   3  bmo financial group  Dead Industry- They will squeeze you like a le...   
3   4  bmo financial group  In BMO agile team, one developer has to servic...   
4   5  bmo financial group                                               Good   

  rating                 author     author_status         location  \
0    3.0        Credit Analyst   Current Employee     Burnaby, BC    
1    3.0       Project Manager    Former Employee     Toronto, ON    
2    1.0     Assistant Manager    Former Employee   Cambridge, ON    
3    2.0   Software Specialist    Former Employee     Toronto, ON    
4    5.0  Relationship Manager   Current Employee     Nanaimo, BC    

            date                                        description  \
0   11 July 2018  Not a bad job but can get boring easily. Manag...   
1   7 April 2020  co-workers are friendly but pay is not good at...   
2   6 April 2020  They give you the illusion of how great of a j...   
3   6 April 2020  Basically, in BMO agile team, they hire 5+ per...   
4   4 April 2020  Work/life balance and customer centric. Traini...   

       source  
0  indeed.com  
1  indeed.com  
2  indeed.com  
3  indeed.com  
4  indeed.com

In [23]:
df[['title','company','rating','author','author_status','location','date','source','description']].to_sql('company_review', con=engine, if_exists='append', index=False)

# Utilities

This section composes of all common functions for data analysis tasks

In [7]:
# Extract top n words using CountVectorizer
def get_top_n_words(corpus, stop_words=None, n=None):
    vec = CountVectorizer(stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

# Extract top n bigrams 
def get_top_n_bigram(corpus, stop_words=None, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

# Extract top n bigrams 
def get_top_n_trigram(corpus, stop_words=None, n=None):
    vec = CountVectorizer(ngram_range=(3, 3), stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

# Job Description Analysis

## Find top words in job description before removing stop words

In [8]:
n_top = 20
common_words = get_top_n_words(df['description'], stop_words=None, n=n_top)
df1 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df1.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Red', title='Top 20 words in job-description before removing stop words')

## Find top words in job description after removing stop words

In [9]:
common_words = get_top_n_words(df['description'], stop_words='english', n=n_top)
df2 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df2.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 words in job-description after removing stop words')

## Find top words in job description of data scientist role

In [10]:
common_words = get_top_n_words(df[df['search_kw'] == 'data scientist']['description'], stop_words='english', n=n_top)
df2 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df2.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 words in job-description of data scientist role')

## Find top words in job description of data engineer role

In [11]:
n_top = 20
common_words = get_top_n_words(df[df['search_kw'] == 'data engineer']['description'], stop_words='english', n=n_top)
df2 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df2.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 words in job-description of data engineer role')

## Find top words in job description of data analyst role

In [12]:
n_top = 20
common_words = get_top_n_words(df[df['search_kw'] == 'data analyst']['description'], stop_words='english', n=n_top)
df2 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df2.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 words in job-description of data analyst role')

## Find top bigrams in job description before removing stop words

In [13]:
common_words = get_top_n_bigram(df['description'], stop_words=None, n=n_top)
df1 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df1.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Red', title='Top 20 bigrams in job-description before removing stop words')

## Find top bigrams in job description after removing stop words

In [14]:
common_words = get_top_n_bigram(df['description'], stop_words='english', n=n_top)
df4 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df4.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 bigrams in the job-description after removing stop words')

## Find top bigrams in job description of data scientist role

In [15]:
common_words = get_top_n_bigram(df[df['search_kw'] == 'data scientist']['description'], stop_words='english', n=n_top)
df1 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df1.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 bigrams in the job-description of data scientist role')

## Find top bigrams in job description of data engineer role

In [16]:
common_words = get_top_n_bigram(df[df['search_kw'] == 'data engineer']['description'], stop_words='english', n=n_top)
df2 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df2.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 bigrams in the job-description of data engineer role')

## Find top bigrams in job description of data analyst role

In [17]:
common_words = get_top_n_bigram(df[df['search_kw'] == 'data analyst']['description'], stop_words='english', n=n_top)
df3 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df3.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 bigrams in the job-description of data analyst role')

## Save all top bigrams 

In [18]:
df1['search_kw'] = 'data scientist'
df2['search_kw'] = 'data engineer'
df3['search_kw'] = 'data analyst'
result = pd.concat([df1, df2, df3])
# result['description'].unique()
result.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top bigrams in the job-description of data scientist, engineer and analyst role')
result.to_csv('results/top_bigrams.csv', index=False)

## Find top trigrams in job description before removing stop words

In [19]:
common_words = get_top_n_trigram(df['description'], stop_words=None, n=n_top)
df1 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df1.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Red', title='Top 20 trigrams in job-description before removing stop words')

## Find top trigrams in job description after removing stop words

In [20]:
common_words = get_top_n_trigram(df['description'], stop_words='english', n=n_top)
df4 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df4.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 trigrams in the job-description after removing stop words')

## Find top trigrams in the job description of data scientist role

In [21]:
common_words = get_top_n_trigram(df[df['search_kw']=='data scientist']['description'], stop_words='english', n=n_top)
df4 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df4.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 trigrams in the job-description of data scientist role')

## Find top trigrams in the job description of data engineer role

In [23]:
common_words = get_top_n_trigram(df[df['search_kw']=='data engineer']['description'], stop_words='english', n=n_top)
df4 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df4.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 trigrams in the job-description of data engineer role')

## Find top trigrams in the job description of data analyst role

In [24]:
common_words = get_top_n_trigram(df[df['search_kw']=='data analyst']['description'], stop_words='english', n=n_top)
df4 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df4.groupby('description').sum()['count'].sort_values(ascending=False).iplot(
    kind='bar', yTitle='Count', linecolor='black',colors='Green', title='Top 20 trigrams in the job-description of data analyst role')